In [1]:
from catalog import Catalog
rowan_catalog = Catalog(term='spring 2024')

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


/Users/tarosh/Library/CloudStorage/OneDrive-Personal/Projects/mnist/data/MNIST/rowan_courses/catalog.py:2: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm
  8%|▊         | 11/145 [00:41<08:23,  3.75s/it]


AttributeError: 'float' object has no attribute 'split'

In [3]:
import re

# thanks to bing chat
class PreqParser():
    """parses prerequisites from Rowan's detailed course information
    """
    def __init__(self, inp: str) -> None:
        self.inp = inp

        self.tokens = []
        tokens = re.findall(r'([A-Z]{2,4} \d{5}|\(|\)|or|and)', inp)
        for i in range(len(tokens)):
            if tokens[i] == '(' and tokens[i+1] == ')':
                continue
            if tokens[i] == ')' and tokens[i-1] == '(':
                continue
            self.tokens.append(tokens[i])

        # abstract syntax tree
        self.ast = self.parse()

    def parse(self):
        index, ast = self._parse_or(0)
        if index != len(self.tokens):
            raise ValueError(f"Unexpected tokens at the end: {self.inp}")
        return ast

    def _parse_expression(self, index):
        if self.tokens[index] == '(':
            index, node = self._parse_or(index + 1)
            if self.tokens[index] != ')':
                raise ValueError(f"Mismatched parentheses: {self.inp}")
            return index + 1, node
        else:
            return index + 1, self.tokens[index]

    def _parse_and(self, index):
        index, left = self._parse_expression(index)
        while index < len(self.tokens) and self.tokens[index] == 'and':
            index, right = self._parse_expression(index + 1)
            left = ('and', left, right)
        return index, left

    def _parse_or(self, index):
        index, left = self._parse_and(index)
        while index < len(self.tokens) and self.tokens[index] == 'or':
            index, right = self._parse_and(index + 1)
            left = ('or', left, right)
        return index, left

    def evaluate(self, context: dict) -> bool:
        """returns if preqs are vaild, given the courses taken
        Args:
            context: dict where keys are courses, and values is boolean
                example - context = {
                                        'CS 10337': True,
                                        'MIS 02337': False,
                                        'CS 04430': True,
                                        'CS 02435': False,
                                        'CS 10338': True,
                                        'CS 10339': True
                                    }
        Returns:
            True or False of if the context is qualified for this course preq
        """
        if isinstance(self.ast, str):
            return context.get(self.ast, False)
        op, left, right = self.ast
        if op == 'and':
            return self.evaluate(left, context) and self.evaluate(right, context)
        elif op == 'or':
            return self.evaluate(left, context) or self.evaluate(right, context)
        else:
            raise ValueError(f"Unknown operator: {op}")

In [9]:
test = "Undergraduate level  STAT 02260 Minimum Grade of D- and Undergraduate level  ACC 03211 Minimum Grade of D- and (Undergraduate level  STAT 02265 Minimum Grade of D- (may be taken concurrently) or Undergraduate level  STAT 02261 Minimum Grade of D- (may be taken concurrently)) and (Undergraduate level  MATH 03125 Minimum Grade of D- or Undergraduate level  MATH 01130 Minimum Grade of D- or Undergraduate level  MATH 01140 Minimum Grade of D-)"
test2 = "FOUNDATIONS OF ACCTG 2 or ( PRINCIPLES OF ACCTG I 2 and  PRINCIPLES OF ACCTG II 2) or Undergraduate level  ACC 03405 Minimum Grade of C or (Undergraduate level  ACC 03210 Minimum Grade of C and Undergraduate level  ACC 03211 Minimum Grade of C)"
out = PreqParser(test2)

ValueError: Unexpected tokens at the end: FOUNDATIONS OF ACCTG 2 or ( PRINCIPLES OF ACCTG I 2 and  PRINCIPLES OF ACCTG II 2) or Undergraduate level  ACC 03405 Minimum Grade of C or (Undergraduate level  ACC 03210 Minimum Grade of C and Undergraduate level  ACC 03211 Minimum Grade of C)

In [10]:
out.tokens

['STAT 02260',
 'and',
 'ACC 03211',
 'and',
 '(',
 'STAT 02265',
 'or',
 'STAT 02261',
 ')',
 'and',
 '(',
 'MATH 03125',
 'or',
 'MATH 01130',
 'or',
 'MATH 01140',
 ')']